In [4]:
import csv
import json
import pandas as pd
import numpy as np
import datetime

# Define Read & Write JSON Function

In [8]:
def store(data):
    with open(folder + 'Summary_Guide.json', 'w') as json_file:
        json_file.write(json.dumps(data))

def load():
    with open(folder + 'Summary_Guide.json') as json_file:
        data = json.load(json_file)
        return data

# Find Data Type

In [33]:
def find_type(data, column): 
    if type(data[column][0]) == 'str':
        return 'String'
    else:
        return 'Numeric'

# Import Data Dictionary

In [11]:
import xlrd
data_dic = pd.read_csv(folder + 'biz_dictionary.csv')
data_dic.set_index('label', inplace=True)

# Data Dic Map

In [13]:
dic_type = {}
dic_type['int'] = 'Numeric'
dic_type['tinyint'] = 'Numeric'
dic_type['numeric'] = 'Numeric'
dic_type['varchar'] = 'String'
dic_type['date'] = 'Date'
dic_type['smallint'] = 'Numeric'
dic_type['numeric(5, 2)'] = 'Numeric'
dic_type['DECIMAL(20,4)'] = 'Numeric'

# Locate folder and files

In [50]:
files = ['file1', 'file2']
folder = 'folder'
years = [i.split('.')[0].split('_')[-1] for i in files]

# JSON Initialization

In [51]:
data = {
    'zipped_file': 
    {
        'name': '', 
        'vendor': '',
        'count': '',
        'file_names': [],
        'content': '',
        'how': '',
        'issues': '',
        'invalid_years': [],
        'purpose': ''
    },
    'individual_files': {},
    'across_files': 
    {
        'total_matches':'',
        'total_returns':'',
        "additional_notes":''
    }
}

# Initialize non-calculate section

In [52]:
#input count if non matches are not returned
input_count = 49841

#zipped file section
if min(years) == max(years):
    zip_name = "_".join(files[0].split('_')[:-1] + [min(years)]) + '.zip'
else:
    zip_name = "_".join(files[0].split('_')[:-1] + [min(years)] + [max(years)]) + '.zip'
zip_vendor = files[0].split('_')[0]
zip_count = len(files)
file_names = files

#get from client manager
content = ''
purpose = ''

invalid_years = []
data['zipped_file']['name'] = zip_name
data['zipped_file']['vendor'] = zip_vendor
data['zipped_file']['count'] = zip_count
data['zipped_file']['file_names'] = file_names
data['zipped_file']['content'] = content
data['zipped_file']['purpose'] = purpose
data['zipped_file']['invalid_years'] = invalid_years

#accross file section

across_returns = '-1'
across_notes = ''

data['across_files']['total_returns'] = across_returns
data['across_files']['additional_notes'] = across_notes

# Calculate Each File's Summary

In [53]:
cur_match = []
for file in files:
    d=pd.read_csv(folder+file)
    new_dic={}
    new_dic['column_count']=d.shape[1]
    new_dic['row_count']=d.shape[0]
    
    #columns
    new_dic['columns']={}
    for column in d.columns:
        new_dic['columns'][column]={}
        
        try:
            new_dic['columns'][column]["description"]=data_dic.loc[column.split('.')[-1],'label']+' (description:'\
                                                    +data_dic.loc[column.split('.')[-1],'description'] +')'
            new_dic['columns'][column]['datatype']=dic_type[data_dic.loc[column.split('.')[-1],'datatype']]
        except:
            new_dic['columns'][column]["description"] = ''
            new_dic['columns'][column]['datatype'] = find_type(d,column)
            
        #need to add int to make type consistent
        new_dic['columns'][column]['missing']=int(max(input_count, d.count()['id'])-d.count()[column])
        new_dic['columns'][column]['manipulated']=False
        new_dic['columns'][column]['raw']=True
        new_dic['columns'][column]['additional']=''
        
    new_dic['hit_rate']=max(d.count()[1:])/max(input_count, d.count()['id'])
    new_dic['invalid_columns']=[]
    new_dic['issues']=''
    data['individual_files'][file]=new_dic
    
    #accross file matches
    cur_match.extend(list(d['id']))
data['across_files']['total_matches'] = len(set(cur_match))

# Find Categorical

In [ ]:
categorical_attributes = []
for column in sample.columns:
    if sample[column].count()>5000 and len(sample[column].value_counts())<100 and len(sample[column].value_counts())>1:
        categorical_attributes.append(column)

In [56]:
for file in files:
    for column in categorical_attributes:
        data['individual_files'][file]['columns'][column]['datatype'] = 'Categorical Variables'

# Write JSON Data

In [ ]:
store(data)
data = load()